In [1]:
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
import re
from scipy.stats import spearmanr
import csv

import warnings
warnings.filterwarnings("ignore")

In [2]:
# setting = "wd=0.01_lr=3e-5"
setting = "wd=0.0_lr=5e-5"

In [5]:
def read_nodes(file_path):
    int_list = []
    with open(file_path, 'r') as csvfile:
        csv_reader = csv.reader(csvfile)
        for row in csv_reader:
            for item in row:
                try:
                    int_list.append(int(item))
                except ValueError:
                    print(f"Warning: '{item}' could not be converted to an integer and was skipped.")
    return int_list



def calculate_one(path):
    # score = torch.load(path, map_location=torch.device('cpu'))  # _test_0225_regroup
    score = torch.load(path, map_location=torch.device('cpu'))
    print(score)
    # score = torch.rand(5000, 500)
    print("score shape:", score.shape)

    nodes_str = [f"./checkpoints/{setting}/{i}/train_index.csv" for i in range(50)]
    # for i in range(50):
    #     nodes_str.append(f"./checkpoints/{i}/train_index.csv")

    full_nodes = [i for i in range(4656)]

    node_list = []
    for node_str in nodes_str:
        numbers = read_nodes(node_str)
        index = []
        for number in numbers:
            index.append(full_nodes.index(number))
        node_list.append(index)

    loss_list = torch.load(f"./results/{setting}/gt.pt", map_location=torch.device('cpu')).detach()

    approx_output = []
    for i in range(len(nodes_str)):
        score_approx_0 = score[node_list[i], :]
        sum_0 = torch.sum(score_approx_0, axis=0)
        approx_output.append(sum_0)

    print(len(loss_list), loss_list[0].shape)
    print(len(approx_output), approx_output[0].shape)

    res = 0
    counter = 0
    for i in range(481):
        tmp = spearmanr(np.array([approx_output[k][i] for k in range(len(approx_output))]),
                        np.array([loss_list[k][i].numpy() for k in range(len(loss_list))])).statistic
        if np.isnan(tmp):
            print("Numerical issue")
            continue
        res += tmp
        counter += 1

    print(counter)

    return res/counter, loss_list, approx_output

path = f"results/{setting}/IF/Linear_LayerNorm/default_SJLT_R-64(U)_thrd-0.0_randrop-0.2.pt"
print(torch.load(f"results/{setting}/IF/Linear_LayerNorm/default_SJLT_R-64(U)_thrd-0.0_randrop-0.2_profile.pt"))
print(calculate_one(path)[0])

{'projection': 83.05920958518982, 'gradient': 136.68024682998657, 'hessian': 22.552314281463623, 'inverse_hessian': 15.064489364624023}
tensor([[  72.7343, -123.6624,  -31.7277,  ...,  167.1384,  -36.7731,
         -183.7464],
        [-113.3401,  113.1658,  128.8549,  ...,   35.6288,  160.8482,
           10.7976],
        [ 111.6066,  309.4640,   61.0273,  ...,  -72.9132,   13.6338,
          145.4602],
        ...,
        [ 271.4454,   39.2268,   43.2614,  ...,   60.1536,   86.6858,
         -132.5101],
        [ 242.7752,   58.5168,   49.8840,  ...,  220.3853,  149.6035,
          -22.6613],
        [  27.1471,   31.0507,  286.8348,  ...,  -84.1284,   23.2444,
         -368.3138]])
score shape: torch.Size([4656, 481])
50 torch.Size([481])
50 torch.Size([481])
481
0.16514835899839014
